<a href="https://colab.research.google.com/github/bowenhickman/python_hw/blob/main/FINAL_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install the alpha_vantage package
!pip install alpha_vantage

# import the stuff we need
import pandas as pd
import json
import os
from datetime import datetime
from alpha_vantage.timeseries import TimeSeries

# Your API key and stock tickers
API_KEY = 'YOUR_ALPHA_VANTAGE_KEY'
tickers = ['AAPL', 'GOOG', 'MSFT', 'AMZN', 'TSLA', 'NVDA', 'META', 'ADBE', 'NFLX', 'INTC']

# get the API thing ready
ts = TimeSeries(key=API_KEY, output_format='pandas')

# function to get and save stock data
def get_and_save_stock_data(ticker):
    data, _ = ts.get_daily(symbol=ticker, outputsize='full')
    data = data.rename(columns=lambda col: col[3:].strip())  # get rid of '1. ', '2. ', etc.

    filename = ticker + ".csv"

    if os.path.exists(filename):
        old_data = pd.read_csv(filename, index_col=0, parse_dates=True)
        # combine old and new data
        new_data = pd.concat([data, old_data])
        new_data = new_data[~new_data.index.duplicated(keep='first')]
        new_data = new_data.sort_index()
    else:
        new_data = data.sort_index()

    # save to CSV
    new_data.to_csv(filename)
    return new_data

# strategy 1 - simple moving average
def simple_moving_average(df):
    df['SMA'] = df['close'].rolling(window=5).mean()
    signals = []

    for i in range(1, len(df)):
        today = df['close'].iloc[i]
        sma_today = df['SMA'].iloc[i]
        yesterday = df['close'].iloc[i - 1]
        sma_yesterday = df['SMA'].iloc[i - 1]

        if today > sma_today and yesterday <= sma_yesterday:
            signals.append(('buy', df.index[i]))
        elif today < sma_today and yesterday >= sma_yesterday:
            signals.append(('sell', df.index[i]))

    return signals

# strategy 2 - mean reversion
def mean_reversion(df):
    df['mean'] = df['close'].rolling(window=20).mean()
    df['std'] = df['close'].rolling(window=20).std()
    upper = df['mean'] + df['std']
    lower = df['mean'] - df['std']

    signals = []
    for i in range(1, len(df)):
        price = df['close'].iloc[i]
        if price < lower.iloc[i]:
            signals.append(('buy', df.index[i]))
        elif price > upper.iloc[i]:
            signals.append(('sell', df.index[i]))

    return signals

# strategy 3 - bollinger band thing
def bollinger_band_strategy(df):
    avg = df['close'].rolling(window=20).mean()
    signals = []

    for i in range(1, len(df)):
        price = df['close'].iloc[i]
        if price > avg.iloc[i] * 1.05:
            signals.append(('buy', df.index[i]))
        elif price < avg.iloc[i] * 0.95:
            signals.append(('sell', df.index[i]))

    return signals

# calculate how much money made or lost
def calculate_profit(df, signals):
    profit = 0
    bought_price = 0
    holding = False

    for sig in signals:
        action = sig[0]
        date = sig[1]
        price = df.loc[date]['close']

        if action == 'buy' and not holding:
            bought_price = price
            holding = True
        elif action == 'sell' and holding:
            profit += price - bought_price
            holding = False

    return profit

# dictionary to hold all results
results = {}

# loop through the tickers
for ticker in tickers:
    df = get_and_save_stock_data(ticker)

    sma = simple_moving_average(df)
    mr = mean_reversion(df)
    bb = bollinger_band_strategy(df)

    sma_profit = calculate_profit(df, sma)
    mr_profit = calculate_profit(df, mr)
    bb_profit = calculate_profit(df, bb)

    results[ticker] = {
        'SMA Profit': sma_profit,
        'Mean Reversion Profit': mr_profit,
        'Bollinger Band Profit': bb_profit
    }

    # check if last signal is from the last row
    for strat_name, strat_signals in zip(['SMA', 'MeanReversion', 'Bollinger'], [sma, mr, bb]):
        if len(strat_signals) > 0 and strat_signals[-1][1] == df.index[-1]:
            print("You should", strat_signals[-1][0], ticker, "today based on", strat_name)

# write results to JSON file
with open('results.json', 'w') as f:
    json.dump(results, f, indent=4)

# figure out best one
best = {'ticker': '', 'strategy': '', 'profit': -999999}

for t in results:
    for strat in results[t]:
        if results[t][strat] > best['profit']:
            best['ticker'] = t
            best['strategy'] = strat
            best['profit'] = results[t][strat]

print("Most profitable strategy:")
print(best)


You should sell GOOG today based on MeanReversion
You should buy GOOG today based on Bollinger
You should sell MSFT today based on MeanReversion
You should sell TSLA today based on MeanReversion
You should buy TSLA today based on Bollinger
You should buy NVDA today based on Bollinger
You should sell NFLX today based on MeanReversion
You should buy NFLX today based on Bollinger
You should sell INTC today based on SMA
Most profitable strategy:
{'ticker': 'GOOG', 'strategy': 'Mean Reversion Profit', 'profit': np.float64(1351.585000000001)}
